<a href="https://colab.research.google.com/github/seeedata/BADA/blob/main/Project/1102%20TheNumbers%20Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import pandas as pd
import numpy as np
import calendar
import random

# 정리

- 전체 달력 페이지

https://www.the-numbers.com/box-office/2020

- 주별 순위 페이지

https://www.the-numbers.com/box-office-chart/weekly/2020/01/03   => 뒤쪽 수정

- rank

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td:nth-child(1) )    
    
    => 첫번째 child 안의 숫자가 변함

- title

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td:nth-child(3) > b > a )    
    
    => 첫번째 child 안의 숫자가 변함

- gross

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td.data.sorting_1 )

- theaters

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td:nth-child(7) )

- per theater

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td.data.chart_grey )

- total gross

    ( #box_office_weekly_table > tbody > tr:nth-child(1) > td:nth-child(10) )

# url 생성

In [ ]:
def make_url():
    url_list = []

    for month in range(1, 13):
        # 해당 월의 마지막 일자를 구합니다
        last_day = calendar.monthrange(2020, month)[1]

        for date in range(1, last_day + 1):
            url = f"https://www.the-numbers.com/box-office-chart/weekly/2020/{month}/{date}"
            url_list.append(url)

    return url_list

In [ ]:
url_list = make_url(); url_list

['https://www.the-numbers.com/box-office-chart/weekly/2020/1/1',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/2',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/3',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/4',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/5',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/6',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/7',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/8',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/9',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/10',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/11',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/12',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/13',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/14',
 'https://www.the-numbers.com/box-office-chart/weekly/2020/1/15',
 'https://www.the-n

# 크롤링!

In [ ]:
def crawling(index):
    url = url_list[index]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Whale/3.23.214.10 Safari/537.36'}

    df = pd.DataFrame(columns=['ranks', 'titles', 'index', 'gross', 'theaters', 'per_theater', 'total_gross'])

    try:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        for i in range(1, 21):  #Top 20까지 추출
            rank = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td:nth-child(1)")
            rank_str = rank[0].text if rank else None

            title = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td:nth-child(3) > b > a")
            title_str = title[0].text if title else None

            gross = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td:nth-child(5)")
            gross_str = gross[0].text if gross else None

            theaters = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td:nth-child(7)")
            theaters_str = theaters[0].text if theaters else None

            per_theater = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td.data.chart_grey")
            per_theater_str = per_theater[0].text if per_theater else None

            total_gross = soup.select(f"#box_office_weekly_table > tbody > tr:nth-child({i}) > td:nth-child(10)")
            total_gross_str = total_gross[0].text if total_gross else None


            df = pd.concat([df, pd.DataFrame({'ranks': [rank_str], 'titles': [title_str], 'index': [index],
                                              'gross': [gross_str], 'theaters': [theaters_str], 'per_theater': [per_theater_str], 'total_gross': [total_gross_str]})], ignore_index=True)

            time.sleep(random.randint(1, 4))

    except Exception as e:
        print(f"An error occurred: {e}")

    return df

In [ ]:
whole_df = pd.DataFrame()

for index in tqdm(range(2, len(url_list), 7)):
    df = crawling(index)
    whole_df = pd.concat([whole_df, df])

whole_df

  0%|          | 0/52 [00:00<?, ?it/s]

,ranks,titles,index,gross,theaters,per_theater,total_gross
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690"
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981"
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009"
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042"
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715"
...,...,...,...,...,...,...,...
15,-,All My Life,359,"$23,840",133,$179,"$899,585"
16,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864"
17,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940"
18,-,Archenemy,359,"$11,003",2,"$5,502","$96,828"


In [ ]:
whole_df.to_csv("C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/영화1.csv", index=False)

In [ ]:
data = whole_df.copy()

In [ ]:
data.isna().sum()

ranks          231
titles         231
index            0
gross          231
theaters       231
per_theater    231
total_gross    231
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1040 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ranks        809 non-null    object
 1   titles       809 non-null    object
 2   index        1040 non-null   object
 3   gross        809 non-null    object
 4   theaters     809 non-null    object
 5   per_theater  809 non-null    object
 6   total_gross  809 non-null    object
dtypes: object(7)
memory usage: 65.0+ KB


In [ ]:
data['ranks'].value_counts()

ranks
-     441
1      44
2      36
3      32
4      30
5      26
6      20
7      17
8      15
9      14
11     13
10     13
13     12
14     12
15     12
16     12
17     12
18     12
19     12
20     12
12     12
Name: count, dtype: int64

In [ ]:
data['index'].value_counts()

index
2      20
9      20
198    20
205    20
212    20
219    20
226    20
233    20
240    20
247    20
254    20
261    20
268    20
275    20
282    20
289    20
296    20
303    20
310    20
317    20
324    20
331    20
338    20
345    20
352    20
191    20
184    20
177    20
86     20
16     20
23     20
30     20
37     20
44     20
51     20
58     20
65     20
72     20
79     20
93     20
170    20
100    20
107    20
114    20
121    20
128    20
135    20
142    20
149    20
156    20
163    20
359    20
Name: count, dtype: int64

In [ ]:
len(data)

1040

In [ ]:
data

,ranks,titles,index,gross,theaters,per_theater,total_gross
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690"
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981"
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009"
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042"
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715"
...,...,...,...,...,...,...,...
15,-,All My Life,359,"$23,840",133,$179,"$899,585"
16,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864"
17,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940"
18,-,Archenemy,359,"$11,003",2,"$5,502","$96,828"


In [ ]:
data = data.reset_index(drop=True)
data

,ranks,titles,index,gross,theaters,per_theater,total_gross
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690"
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981"
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009"
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042"
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715"
...,...,...,...,...,...,...,...
1035,-,All My Life,359,"$23,840",133,$179,"$899,585"
1036,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864"
1037,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940"
1038,-,Archenemy,359,"$11,003",2,"$5,502","$96,828"


In [ ]:
# None 값을 가진 행 제거
rows_to_remove = []
for row in range(len(data)):
    if (data.loc[row, 'ranks'] is None) and (data.loc[row, 'titles'] is None):
        rows_to_remove.append(row)

# 특정 행 삭제
data = data.drop(rows_to_remove)

# 인덱스를 리셋하여 삭제된 행의 인덱스를 정리
data = data.reset_index(drop=True)
data

,ranks,titles,index,gross,theaters,per_theater,total_gross
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690"
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981"
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009"
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042"
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715"
...,...,...,...,...,...,...,...
804,-,All My Life,359,"$23,840",133,$179,"$899,585"
805,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864"
806,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940"
807,-,Archenemy,359,"$11,003",2,"$5,502","$96,828"


In [ ]:
data.to_csv('C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/영화크롤링최종.csv', index=False)